In [37]:
# This cell performs data processing or transformation
import pandas as pd
import numpy as np

In [51]:
# This cell performs data processing or transformation
historical_df = pd.read_csv('historical_data.csv')
fear_greed = pd.read_csv('fear_greed_index.csv')

In [53]:
# This cell performs data processing or transformation
historical_df.head()

,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,02-12-2024 22:50,0.000000,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.345404,8.950000e+14,1.730000e+12
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,BUY,02-12-2024 22:50,986.524596,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.005600,4.430000e+14,1.730000e+12
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,BUY,02-12-2024 22:50,1002.518996,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050431,6.600000e+14,1.730000e+12
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,BUY,02-12-2024 22:50,1146.558564,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050043,1.080000e+15,1.730000e+12
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,BUY,02-12-2024 22:50,1289.488521,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.003055,1.050000e+15,1.730000e+12


In [55]:
# This cell performs data processing or transformation
historical_df.Account.nunique()
#there are 32 different traders

32

In [57]:
# This cell performs data processing or transformation
historical_df.drop(['Transaction Hash','Order ID','Trade ID','Timestamp'],axis = 1,inplace = True)

### Renaming Columns for better Interpretability

In [61]:
# This cell performs data processing or transformation
cols = historical_df.columns
cols

Index(['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side',
       'Timestamp IST', 'Start Position', 'Direction', 'Closed PnL', 'Crossed',
       'Fee'],
      dtype='object')

In [63]:
# This cell performs data processing or transformation
historical_df.rename(columns = {'Side':'Buy/Sell','Size USD':'Total_Amount','Closed PnL':'PnL',
                   'Account':'User_Id','Start Position':'Start_Position','Execution Price':'Execution_Price','Size Tokens':'Size_Tokens',
                   'Timestamp IST':'Timestamp'},inplace=True)

In [65]:
# This cell performs data processing or transformation
historical_df.columns

Index(['User_Id', 'Coin', 'Execution_Price', 'Size_Tokens', 'Total_Amount',
       'Buy/Sell', 'Timestamp', 'Start_Position', 'Direction', 'PnL',
       'Crossed', 'Fee'],
      dtype='object')

In [67]:
# This cell performs data processing or transformation
historical_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211224 entries, 0 to 211223
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   User_Id          211224 non-null  object 
 1   Coin             211224 non-null  object 
 2   Execution_Price  211224 non-null  float64
 3   Size_Tokens      211224 non-null  float64
 4   Total_Amount     211224 non-null  float64
 5   Buy/Sell         211224 non-null  object 
 6   Timestamp        211224 non-null  object 
 7   Start_Position   211224 non-null  float64
 8   Direction        211224 non-null  object 
 9   PnL              211224 non-null  float64
 10  Crossed          211224 non-null  bool   
 11  Fee              211224 non-null  float64
dtypes: bool(1), float64(6), object(5)
memory usage: 17.9+ MB


In [69]:
# This cell performs data processing or transformation
 def transform(df):
    df = df.copy()
    
    # 1. Coin to lowercase and category
    
    df['Coin'] = df['Coin'].str.lower()
    df['Coin'] = df['Coin'].astype('category')
    
    # 2. Buy/Sell encoding + category

    df['Buy/Sell'] = df['Buy/Sell'].str.strip().str.lower()
    df['Buy/Sell'] = df['Buy/Sell'].astype('category')
    
    # 3. Crossed: True/False to 1/0
    df['Crossed'] = df['Crossed'].astype('bool')
    
    # 4. Timestamp: Convert to datetime

    df['Timestamp'] = pd.to_datetime(df['Timestamp'],format="%d-%m-%Y %H:%M")

    return df    

historical_df = transform(historical_df)



In [71]:
# This cell performs data processing or transformation
historical_df['Date'] = historical_df['Timestamp'].dt.date

In [73]:
# 5. Fix Direction
def fix_direction(row):
    side = row['Buy/Sell']
    pnl = row['PnL']
    if side == 'buy':  # Buy
        return "open long" if pnl == 0 else "close short"
    elif side == 'sell':  # Sell
        return "open short" if pnl == 0 else "close long"
    else:
        return "Unknown"
    
historical_df.loc[:,'Direction'] =  historical_df.apply(fix_direction, axis=1)
historical_df['Direction'] = historical_df['Direction'].astype('category')

In [75]:
# This cell performs data processing or transformation
def label_time_of_day(df):

    df = df.copy()
    # Create labeled column using hour
    return pd.cut(
        df['Timestamp'].dt.hour,
        bins=[-1, 6, 11, 17, 21, 24],
        labels=['lateNight', 'morning', 'afternoon', 'evening', 'night'],
        right=True
    )

    

temp = pd.DataFrame({"TimeofDay":label_time_of_day(historical_df)})
#adding TimeofDay column at specfic position
historical_df = pd.concat([historical_df.iloc[:, :7], temp, historical_df.iloc[:, 7:]], axis=1)

In [77]:
# This cell performs data processing or transformation
historical_df.head()

,User_Id,Coin,Execution_Price,Size_Tokens,Total_Amount,Buy/Sell,Timestamp,TimeofDay,Start_Position,Direction,PnL,Crossed,Fee,Date
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,buy,2024-12-02 22:50:00,night,0.000000,open long,0.0,True,0.345404,2024-12-02
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,buy,2024-12-02 22:50:00,night,986.524596,open long,0.0,True,0.005600,2024-12-02
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,buy,2024-12-02 22:50:00,night,1002.518996,open long,0.0,True,0.050431,2024-12-02
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,buy,2024-12-02 22:50:00,night,1146.558564,open long,0.0,True,0.050043,2024-12-02
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,buy,2024-12-02 22:50:00,night,1289.488521,open long,0.0,True,0.003055,2024-12-02


In [79]:
# This cell performs data processing or transformation
historical_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211224 entries, 0 to 211223
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   User_Id          211224 non-null  object        
 1   Coin             211224 non-null  category      
 2   Execution_Price  211224 non-null  float64       
 3   Size_Tokens      211224 non-null  float64       
 4   Total_Amount     211224 non-null  float64       
 5   Buy/Sell         211224 non-null  category      
 6   Timestamp        211224 non-null  datetime64[ns]
 7   TimeofDay        211224 non-null  category      
 8   Start_Position   211224 non-null  float64       
 9   Direction        211224 non-null  category      
 10  PnL              211224 non-null  float64       
 11  Crossed          211224 non-null  bool          
 12  Fee              211224 non-null  float64       
 13  Date             211224 non-null  object        
dtypes: bool(1), category

## Clening dataset fear_greed

In [81]:
# This cell performs data processing or transformation
fear_greed.isna().sum()

timestamp         0
value             0
classification    0
date              0
dtype: int64

In [83]:
# This cell performs data processing or transformation
fear_greed.head()

,timestamp,value,classification,date
0,1517463000,30,Fear,2018-02-01
1,1517549400,15,Extreme Fear,2018-02-02
2,1517635800,40,Fear,2018-02-03
3,1517722200,24,Extreme Fear,2018-02-04
4,1517808600,11,Extreme Fear,2018-02-05


In [85]:
# This cell performs data processing or transformation
fear_greed.drop('timestamp',axis=1,inplace=True)

In [89]:
# This cell performs data processing or transformation
fear_greed['date'] = pd.to_datetime(fear_greed['date']).dt.date

In [91]:
# This cell performs data processing or transformation
fear_greed['classification'] = fear_greed['classification'].str.strip().str.lower().astype('category')

In [93]:
# This cell performs data processing or transformation
fear_greed.head()

,value,classification,date
0,30,fear,2018-02-01
1,15,extreme fear,2018-02-02
2,40,fear,2018-02-03
3,24,extreme fear,2018-02-04
4,11,extreme fear,2018-02-05


In [95]:
# This cell performs data processing or transformation
fear_greed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2644 entries, 0 to 2643
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   value           2644 non-null   int64   
 1   classification  2644 non-null   category
 2   date            2644 non-null   object  
dtypes: category(1), int64(1), object(1)
memory usage: 44.2+ KB


In [99]:
# This cell performs data processing or transformation
fear_greed.columns

Index(['value', 'classification', 'date'], dtype='object')

In [101]:
# This cell performs data processing or transformation
fear_greed.rename(columns = {'value':'Setinment_Index','classification':'Fear/Greed','date':'Date'},inplace=True)

In [103]:
# This cell performs data processing or transformation
fear_greed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2644 entries, 0 to 2643
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   Setinment_Index  2644 non-null   int64   
 1   Fear/Greed       2644 non-null   category
 2   Date             2644 non-null   object  
dtypes: category(1), int64(1), object(1)
memory usage: 44.2+ KB


# merging two datasets

In [105]:
# This cell performs data processing or transformation
merged_df = historical_df.merge(fear_greed, on='Date', how='inner')

In [107]:
# This cell performs data processing or transformation
merged_df.isna().sum()

User_Id            0
Coin               0
Execution_Price    0
Size_Tokens        0
Total_Amount       0
Buy/Sell           0
Timestamp          0
TimeofDay          0
Start_Position     0
Direction          0
PnL                0
Crossed            0
Fee                0
Date               0
Setinment_Index    0
Fear/Greed         0
dtype: int64

In [109]:
# This cell performs data processing or transformation
merged_df.head()

,User_Id,Coin,Execution_Price,Size_Tokens,Total_Amount,Buy/Sell,Timestamp,TimeofDay,Start_Position,Direction,PnL,Crossed,Fee,Date,Setinment_Index,Fear/Greed
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,buy,2024-12-02 22:50:00,night,0.000000,open long,0.0,True,0.345404,2024-12-02,80,extreme greed
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,buy,2024-12-02 22:50:00,night,986.524596,open long,0.0,True,0.005600,2024-12-02,80,extreme greed
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,buy,2024-12-02 22:50:00,night,1002.518996,open long,0.0,True,0.050431,2024-12-02,80,extreme greed
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,buy,2024-12-02 22:50:00,night,1146.558564,open long,0.0,True,0.050043,2024-12-02,80,extreme greed
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,buy,2024-12-02 22:50:00,night,1289.488521,open long,0.0,True,0.003055,2024-12-02,80,extreme greed


In [113]:
# This cell performs data processing or transformation
merged_df.shape[0]  / historical_df.shape[0] * 100

99.99715941370299

In [355]:
# means 99% rows are preserved after merging
# this allow us to perform data analysis directly on merged dataset

In [341]:
# This cell performs data processing or transformation
historical_df.to_csv('historical_df_transformed.csv',index=False)

In [343]:
# This cell performs data processing or transformation
fear_greed.to_csv('fear_greed_transformed.csv',index=False)

In [117]:
# This cell performs data processing or transformation
penny_threshold = merged_df['Execution_Price'].quantile(0.25) # Example: coins below the 25th percentile
expensive_threshold = merged_df['Execution_Price'].quantile(0.75) # Example: coins above the 75th percentile

print(f"\nExample thresholds based on quartiles:")
print(f"Penny (<= {penny_threshold:.2f})")
print(f"Moderate (> {penny_threshold:.2f} and <= {expensive_threshold:.2f})")
print(f"Expensive (> {expensive_threshold:.2f})")

# Function to classify coins
def classify_coin_type(price):
    if price <= penny_threshold:
        return 'penny'
    elif price > penny_threshold and price <= expensive_threshold:
        return 'moderate'
    else:
        return 'expensive'

# Apply the function to create the new column
merged_df['Coin_Type'] = merged_df['Execution_Price'].apply(classify_coin_type)


Example thresholds based on quartiles:
Penny (<= 4.86)
Moderate (> 4.86 and <= 101.89)
Expensive (> 101.89)


In [119]:
# This cell performs data processing or transformation
merged_df.head()

,User_Id,Coin,Execution_Price,Size_Tokens,Total_Amount,Buy/Sell,Timestamp,TimeofDay,Start_Position,Direction,PnL,Crossed,Fee,Date,Setinment_Index,Fear/Greed,Coin_Type
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,buy,2024-12-02 22:50:00,night,0.000000,open long,0.0,True,0.345404,2024-12-02,80,extreme greed,moderate
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,buy,2024-12-02 22:50:00,night,986.524596,open long,0.0,True,0.005600,2024-12-02,80,extreme greed,moderate
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,buy,2024-12-02 22:50:00,night,1002.518996,open long,0.0,True,0.050431,2024-12-02,80,extreme greed,moderate
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,buy,2024-12-02 22:50:00,night,1146.558564,open long,0.0,True,0.050043,2024-12-02,80,extreme greed,moderate
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,buy,2024-12-02 22:50:00,night,1289.488521,open long,0.0,True,0.003055,2024-12-02,80,extreme greed,moderate


In [121]:
# Function to categorize trade results based on PnL
def categorize_trade_result(pnl):
    if pnl > 0:
        return 'profit'
    elif pnl < 0:
        return 'loss'
    else:
        return 'nill'

# Apply the function to the 'PnL' column to create the 'TradeResult' column
merged_df['TradeResult'] = merged_df['PnL'].apply(categorize_trade_result)

In [123]:
# Ensure Timestamp is datetime
# Extract hour of the day
merged_df['Timestamp'] = pd.to_datetime(merged_df['Timestamp'],format="%Y-%m-%d %H:%M:%S")
merged_df['Hour'] = merged_df['Timestamp'].dt.hour
merged_df['DayOfWeek'] = merged_df['Timestamp'].dt.day_name()

In [125]:
# This cell performs data processing or transformation
merged_df.head()

,User_Id,Coin,Execution_Price,Size_Tokens,Total_Amount,Buy/Sell,Timestamp,TimeofDay,Start_Position,Direction,PnL,Crossed,Fee,Date,Setinment_Index,Fear/Greed,Coin_Type,TradeResult,Hour,DayOfWeek
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,buy,2024-12-02 22:50:00,night,0.000000,open long,0.0,True,0.345404,2024-12-02,80,extreme greed,moderate,nill,22,Monday
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,buy,2024-12-02 22:50:00,night,986.524596,open long,0.0,True,0.005600,2024-12-02,80,extreme greed,moderate,nill,22,Monday
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,buy,2024-12-02 22:50:00,night,1002.518996,open long,0.0,True,0.050431,2024-12-02,80,extreme greed,moderate,nill,22,Monday
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,buy,2024-12-02 22:50:00,night,1146.558564,open long,0.0,True,0.050043,2024-12-02,80,extreme greed,moderate,nill,22,Monday
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,buy,2024-12-02 22:50:00,night,1289.488521,open long,0.0,True,0.003055,2024-12-02,80,extreme greed,moderate,nill,22,Monday


In [131]:
# This cell performs data processing or transformation
merged_df['Coin_Type'] = merged_df['Coin_Type'].astype('category')
merged_df['TradeResult'] = merged_df['TradeResult'].astype('category')
merged_df['DayOfWeek'] = merged_df['DayOfWeek'].astype('category')

In [133]:
# This cell performs data processing or transformation
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211218 entries, 0 to 211217
Data columns (total 20 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   User_Id          211218 non-null  object        
 1   Coin             211218 non-null  category      
 2   Execution_Price  211218 non-null  float64       
 3   Size_Tokens      211218 non-null  float64       
 4   Total_Amount     211218 non-null  float64       
 5   Buy/Sell         211218 non-null  category      
 6   Timestamp        211218 non-null  datetime64[ns]
 7   TimeofDay        211218 non-null  category      
 8   Start_Position   211218 non-null  float64       
 9   Direction        211218 non-null  category      
 10  PnL              211218 non-null  float64       
 11  Crossed          211218 non-null  bool          
 12  Fee              211218 non-null  float64       
 13  Date             211218 non-null  object        
 14  Setinment_Index  211

In [137]:
# This cell performs data processing or transformation
merged_df.to_csv('Final_df.csv',index=False)

In [ ]:
# This cell performs data processing or transformation
merged_df